# Arkadiusz Pytlik

 ### Połączenie z bazą

In [1]:
import psycopg2 as pg
import pandas as pd

connection = pg.connect(host='pgsql-196447.vipserv.org', port=5432, dbname='wbauer_adb', user='wbauer_adb', password='adb2020')


 ### Zadania wprowadzające
 
 #####  1. Znajdź listę wszystkich filmów o tej samej długości.

In [2]:
length = 106 

df = pd.read_sql(f"SELECT * FROM film WHERE film.length = {length}", con=connection)
df

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,62,Bed Highball,A Astounding Panorama of a Lumberjack And a Do...,2006,1,5,2.99,106,23.99,NC-17,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Deleted Scenes]",'abandon':19 'astound':4 'bed':1 'dog':11 'fun...
1,204,Dalmations Sweden,A Emotional Epistle of a Moose And a Hunter wh...,2006,1,4,0.99,106,25.99,PG,2013-05-26 14:50:58.951,"[Trailers, Commentaries, Deleted Scenes]",'dalmat':1 'emot':4 'epistl':5 'hunter':11 'ma...
2,220,Deer Virginian,A Thoughtful Story of a Mad Cow And a Womanize...,2006,1,7,2.99,106,13.99,NC-17,2013-05-26 14:50:58.951,[Deleted Scenes],"'cow':9 'deer':1 'georgia':21 'mad':8,17 'must..."
3,351,Gaslight Crusade,A Amazing Epistle of a Boy And a Astronaut who...,2006,1,4,2.99,106,10.99,PG,2013-05-26 14:50:58.951,"[Trailers, Deleted Scenes]",'amaz':4 'astronaut':11 'boy':8 'crusad':2 'ep...
4,699,Private Drop,A Stunning Story of a Technical Writer And a H...,2006,1,7,4.99,106,26.99,PG,2013-05-26 14:50:58.951,[Trailers],'agent':18 'baloon':21 'drop':2 'hunter':12 'm...
5,757,Sagebrush Clueless,A Insightful Story of a Lumberjack And a Hunte...,2006,1,4,2.99,106,28.99,G,2013-05-26 14:50:58.951,[Trailers],'ancient':18 'boy':16 'clueless':2 'hunter':11...


W poleceniu nie sprecyzowano długości, zatem wybrałem 106.

##### 2. Znajdź wszystkich klientów mieszkających w tym samym mieście.

In [3]:
city = 'London'

df = pd.read_sql("SELECT city.city, customer.* FROM city " + 
                  "INNER JOIN address ON city.city_id = address.city_id " + 
                  "INNER JOIN customer ON address.address_id = customer.address_id " +
                  "WHERE city.city = '{a}'".format(a=city), con=connection)
df

,city,customer_id,store_id,first_name,last_name,email,address_id,activebool,create_date,last_update,active
0,London,252,2,Mattie,Hoffman,mattie.hoffman@sakilacustomer.org,256,True,2006-02-14,2013-05-26 14:49:45.738,1
1,London,512,1,Cecil,Vines,cecil.vines@sakilacustomer.org,517,True,2006-02-14,2013-05-26 14:49:45.738,1


W poleceniu nie sprecyzowano miasta, zatem wybrałem London.

##### 3. Oblicz średni koszt wypożyczenia wszystkich filmów.

In [4]:
df = pd.read_sql("SELECT AVG(payment.amount) FROM film " + 
                 "INNER JOIN inventory ON film.film_id = inventory.film_id " + 
                 "INNER JOIN rental ON inventory.inventory_id = rental.inventory_id " +
                 "INNER JOIN payment ON rental.rental_id = payment.rental_id ", con=connection)
df

,avg
0,4.200606


##### 4. Oblicz i wyświetl liczbę filmów we wszystkich kategoriach.

In [5]:
df = pd.read_sql("SELECT category.name category, COUNT(category.name) FROM film " +
                 "INNER JOIN film_category ON film.film_id = film_category.film_id " +
                 "INNER JOIN category ON film_category.category_id = category.category_id " +
                 "GROUP BY category.name ", con=connection)
        
df

,category,count
0,Horror,56
1,Comedy,58
2,Sci-Fi,61
3,Drama,62
4,Foreign,73
5,Classics,57
6,Games,61
7,New,63
8,Travel,57
9,Music,51


##### 5. Wyświetl liczbę wszystkich klientów pogrupowanych według kraju.

In [6]:
df = pd.read_sql("SELECT country.country, COUNT(customer.customer_id) FROM country " +
                 "INNER JOIN city ON country.country_id = city.country_id " +
                 "INNER JOIN address ON city.city_id = address.city_id " +
                 "INNER JOIN customer ON address.address_id = customer.address_id " +
                 "GROUP BY country.country ORDER BY country.country", con=connection)
        
df

,country,count
0,Afghanistan,1
1,Algeria,3
2,American Samoa,1
3,Angola,2
4,Anguilla,1
...,...,...
103,Vietnam,6
104,"Virgin Islands, U.S.",1
105,Yemen,4
106,Yugoslavia,2


##### 6. Wyświetl informacje o sklepie, który ma więcej niż 100 klientów i mniej niż 300 klientów.

In [7]:
df = pd.read_sql("SELECT store.* FROM store " +
                 "INNER JOIN customer ON store.store_id = customer.store_id " +
                 "GROUP BY store.store_id " + 
                 "HAVING COUNT(customer.customer_id) > 100 AND COUNT(customer.customer_id) < 300", con=connection)
        
df

,store_id,manager_staff_id,address_id,last_update
0,2,2,2,2006-02-15 09:57:12


##### 7. Wybierz wszystkich klientów, którzy oglądali filmy ponad 200 godzin.

In [8]:
df = pd.read_sql("SELECT customer.first_name, customer.last_name, SUM(film.length) FROM film " +
                 "INNER JOIN inventory ON film.film_id = inventory.film_id " +
                 "INNER JOIN rental ON inventory.inventory_id = rental.inventory_id " +
                 "INNER JOIN customer ON rental.customer_id = customer.customer_id " +
                 "GROUP BY customer.first_name, customer.last_name " +
                 "HAVING SUM(film.length) > 200" +
                 "ORDER BY SUM(film.length), customer.last_name, customer.first_name", con=connection)

df

,first_name,last_name,sum
0,Brian,Wyman,1265
1,Antonio,Meek,1451
2,Leona,Obrien,1588
3,Katherine,Rivera,1615
4,Tiffany,Jordan,1667
...,...,...,...
594,Wesley,Bull,4808
595,Clara,Shaw,4808
596,Tammy,Sanders,5065
597,Eleanor,Hunt,5360


Czasy są podawane w minutach, więc albo zakładamy, że w poleceniu powinno być ponad "200 minut" i wypisujemy wszystkie osoby, lub jeśli przeliczymy 200h na minuty otrzymamy 12000, zatem nikt nie oglądał tak długo filmów.

##### 8. Oblicz średnią wartość wypożyczenia filmu.

In [9]:
# to samo co punkt 3?

##### 9. Oblicz średnią wartość długości filmu we wszystkich kategoriach.

In [10]:
df = pd.read_sql("SELECT category.name category, AVG(film.length) FROM film " +
                 "INNER JOIN film_category ON film.film_id = film_category.film_id " +
                 "INNER JOIN category ON film_category.category_id = category.category_id " +
                 "GROUP BY category.name ", con=connection)
        
df

,category,avg
0,Horror,112.482143
1,Comedy,115.827586
2,Sci-Fi,108.196721
3,Drama,120.838710
4,Foreign,121.698630
5,Classics,111.666667
6,Games,127.836066
7,New,111.126984
8,Travel,113.315789
9,Music,113.647059


##### 10. Znajdź najdłuższe tytuły filmowe we wszystkich kategoriach.

In [11]:
df = pd.read_sql("SELECT category.name category, MAX(film.length) max_length FROM film " +
                 "INNER JOIN film_category ON film.film_id = film_category.film_id " +
                 "INNER JOIN category ON film_category.category_id = category.category_id " +
                 "GROUP BY category.name ", con=connection)
        
df

,category,max_length
0,Horror,181
1,Comedy,185
2,Sci-Fi,185
3,Drama,181
4,Foreign,184
5,Classics,184
6,Games,185
7,New,183
8,Travel,185
9,Music,185


##### 11. Znajdź najdłuższy film we wszystkich kategoriach. Porównaj wynik z pkt 10.

In [12]:
df = pd.read_sql("SELECT * FROM film ORDER BY film.length DESC LIMIT 1" , con=connection)
df

,film_id,title,description,release_year,language_id,rental_duration,rental_rate,length,replacement_cost,rating,last_update,special_features,fulltext
0,141,Chicago North,A Fateful Yarn of a Mad Cow And a Waitress who...,2006,1,6,4.99,185,11.99,PG-13,2013-05-26 14:50:58.951,"[Deleted Scenes, Behind the Scenes]",'battl':15 'california':19 'chicago':1 'cow':9...


Zgadza się to z punktem 10.

 ### Zadania implementacyjne

In [13]:
import main

In [14]:
main.film_in_category(1)

,title,languge,category
0,Amadeus Holy,English,Action
1,American Circus,English,Action
2,Antitrust Tomatoes,English,Action
3,Ark Ridgemont,English,Action
4,Barefoot Manchurian,English,Action
...,...,...,...
59,Uprising Uptown,English,Action
60,Waterfront Deliverance,English,Action
61,Werewolf Lola,English,Action
62,Women Dorado,English,Action


In [15]:
main.number_films_in_category(2)

,category,count
0,Animation,66


In [16]:
main.number_film_by_length(55, 73)

,length,count
0,55,2
1,56,5
2,57,7
3,58,7
4,59,9
5,60,8
6,61,10
7,62,6
8,63,9
9,64,9


In [17]:
main.client_from_city('London')

,city,first_name,last_name
0,London,Mattie,Hoffman
1,London,Cecil,Vines


In [18]:
main.avg_amount_by_length(106)

,length,avg
0,106,3.954405


In [19]:
main.client_by_sum_length(4000)

,first_name,last_name,sum
0,Elsie,Kelley,4014
1,Mike,Way,4023
2,Sherri,Rhodes,4030
3,Regina,Berry,4051
4,Tommy,Collazo,4051
5,Grace,Ellis,4060
6,Jessica,Hall,4150
7,Marsha,Douglas,4154
8,Michelle,Clark,4157
9,Brandon,Huey,4160


In [20]:
main.category_statistic_length('Games')

,category,avg,sum,min,max
0,Games,127.836066,7798,57,185
